In [1]:
import pandas as pd
import re
# from nltk.corpus import words
from nltk.tokenize import word_tokenize
import nltk
import numpy as np
#nltk.download('punkt')
from textblob import Word
pd.options.display.width = 0

import spacy
nlp = spacy.load('en_core_web_sm')
all_stopwords = nlp.Defaults.stop_words

pd.set_option('display.max_rows', 6000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)
#pd.describe_option('display')
# import enchant

In [2]:
def find_in_english_dict(text):
    for w in word_tokenize(text):
        if Word(w.strip().lower()).spellcheck()[0][1]!=1.0 and len(re.findall(r"[A-Z]{2,}", w))>0:
          # print(w)
          return True
    return False

find_in_english_dict('HYPERTHROIDSM')
Word('allergy'.strip().lower()).spellcheck()

[('clergy', 0.48717948717948717),
 ('gallery', 0.28205128205128205),
 ('alley', 0.1282051282051282),
 ('allegro', 0.05128205128205128),
 ('allegory', 0.02564102564102564),
 ('allege', 0.02564102564102564)]

In [3]:
l = {'text_id':['A1','A2','A3','A4'], 'text':['a','b','c', 'd'], 'cui_l':[[1,2,5],[2,5], [],[]],
    'cui_g':[[1,2,3],[2,3],[4],[]], 'cui_d':[[1,2,4],[3], [], []]}
df = pd.DataFrame(data=l)
df['cui_len']=df['cui_l'].apply(lambda x: len(x))
df['cui_g_l']=(df['cui_g']+df['cui_d']).apply(lambda x: list(set(x)))
df['ID'] = ['ID'+ str(x) for x in df.index.tolist()]
df['cui_final'] = np.where(df['cui_len']!=0, df['cui_l'], df['cui_g_l'])
df_id_cui = df[['ID','cui_final']]
df, df_id_cui.explode('cui_final', ignore_index=True), 

(  text_id text      cui_l      cui_g      cui_d  cui_len       cui_g_l   ID  \
 0      A1    a  [1, 2, 5]  [1, 2, 3]  [1, 2, 4]        3  [1, 2, 3, 4]  ID0   
 1      A2    b     [2, 5]     [2, 3]        [3]        2        [2, 3]  ID1   
 2      A3    c         []        [4]         []        0           [4]  ID2   
 3      A4    d         []         []         []        0            []  ID3   
 
    cui_final  
 0  [1, 2, 5]  
 1     [2, 5]  
 2        [4]  
 3         []  ,
     ID cui_final
 0  ID0         1
 1  ID0         2
 2  ID0         5
 3  ID1         2
 4  ID1         5
 5  ID2         4
 6  ID3       NaN)

In [4]:
# merging new text translation
"""path='/nfs/home/vyasa/projects/proj_off/data_off/clarify/spanish_comorbidity/new/'
path_output = '/nfs/home/vyasa/projects/proj_off/data_off/clarify/spanish_comorbidity/new/output/'

df = pd.read_csv(path+'comorbidities_modified_witht_abbr_new.csv', sep=',', error_bad_lines=True, encoding='utf-8')
df.shape

df_new_text = pd.read_csv(path_output+'nohaving_cuiMEV_done.csv', encoding='utf-8')
display(df_new_text.head(5))

df.loc[df['Value_ID'].isin(df_new_text.Value_ID),'replacement'] = df_new_text['replacement '].values
display(df[df['Value_ID'].isin(df_new_text.Value_ID)].head(10))
display (df[['Value_ID', 'replacement']].head(10))
df.to_csv(path+'comorbidities_modified_witht_abbr_new_new.csv', encoding='utf-8', index=False)"""

,Value_ID,value,replacement
0,A10520,PROBLEMAS VISUALES DESDE 2012,Vision Disorders
1,A10523,PANCREATITIS DE ORIGEN LITIASICO,Pancreatolithiasis
2,A10525,factor 5 de LELDEN,Factor V Leiden
3,A10543,OSTEROPORIS,Osteoporosis
4,A10570,ex heroinÛmano,Drug Dependence


,Value_ID,value,value_witht_abbr,replacement
522,A10520,PROBLEMAS VISUALES DESDE 2012,PROBLEMAS VISUALES DESDE 2012,Vision Disorders
525,A10523,PANCREATITIS DE ORIGEN LITIASICO,PANCREATITIS DE ORIGEN LITIASICO,Pancreatolithiasis
527,A10525,factor 5 de LELDEN,factor 5 de LELDEN,Factor V Leiden
545,A10543,OSTEROPORIS,OSTEROPORIS,Osteoporosis
572,A10570,ex heroinÛmano,ex heroinÛmano,Drug Dependence
605,A10603,Lumbociatalgia derecha,Lumbociatalgia derecha,Sciatica
610,A10608,Alergia al espadrapo,Alergia al espadrapo,Allergy
664,A10662,ex politoxicÛmano,ex politoxicÛmano,Drug Dependence
669,A10667,CERVICOARTROSIS,CERVICOARTROSIS,Cervical spondylosis
686,A10684,COLETIASIS,COLETIASIS,Cholelithiasis


,Value_ID,replacement
0,A10000,Chronic gastritis;
1,A10001,Pectoralis major pleomorphic sarcoma;
2,A10002,Hyperuricemia;
3,A10003,Colon polyp operated;
4,A10004,Aortic stenosis operated;
5,A10005,Dyspnea;
6,A10006,Osteoporosis;
7,A10007,Hysterectomy double adnexectomy;
8,A10008,Subclinical hypothyroidism;
9,A10009,Prostate carcinoma;


In [7]:
import scispacy
import spacy
from scispacy.linking import EntityLinker

nlp = spacy.load("en_core_sci_lg")
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": False, "linker_name": "umls", "filter_for_definitions":False})

doc = nlp("Surgery Appendicitis")
# Let's look at a random entity!
entity = doc.ents[0]
print("Name: ", entity)

/nfs/home/vyasa/software/pkg/miniconda3/envs/scispacy/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/nfs/home/vyasa/software/pkg/miniconda3/envs/scispacy/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
Name:  Surgery Appendicitis
/nfs/home/vyasa/software/pkg/miniconda3/envs/scispacy/lib/python3.7/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating th

In [8]:
print (doc.ents)
linker = nlp.get_pipe("scispacy_linker")
for umls_ent in entity._.kb_ents:
	print(linker.kb.cui_to_entity[umls_ent[0]])

(Surgery Appendicitis,)
